In [61]:
import plotly.express as px
import plotly.graph_objs as go

import numpy as np


In [62]:
TRIANGLE_LENGTH = 5

# triangle length must be odd so edges can connect
assert TRIANGLE_LENGTH % 2 == 1 and TRIANGLE_LENGTH >= 5

In [63]:
# HELPER FUNCTIONS FOR VISUALIZATION
def show_2d_points(points, lines = []):
    x_points = [p[0] for p in points]
    y_points = [p[1] for p in points]
    dot_fig = px.scatter(x=x_points, y=y_points, text=range(len(points)))
    line_figs = []
    for line in lines:
        x_lines = [points[i][0] for i in line]
        y_lines = [points[i][1] for i in line]
        line_figs.append(px.line(x=x_lines, y=y_lines))
    fig = go.Figure(data = sum([x.data for x in [dot_fig] + line_figs], ()))
    fig.update_yaxes(scaleanchor='x', scaleratio=1)
    fig.update_traces(textposition='top center')
    fig.update_layout(
        width=500,
        height=500,
    )

    fig.show()

def show_3d_points(points, lines = [], point_labels = True):
    x_points = [p[0] for p in points]
    y_points = [p[1] for p in points]
    z_points = [p[2] for p in points]
    dot_fig = px.scatter_3d(x=x_points, y=y_points, z=z_points)
    if point_labels:
        dot_fig = px.scatter_3d(x=x_points, y=y_points, z=z_points, text=range(len(points)))
        
    line_figs = []
    for line in lines:
        x_lines = [points[i][0] for i in line]
        y_lines = [points[i][1] for i in line]
        z_lines = [points[i][2] for i in line]
        line_figs.append(px.line_3d(x=x_lines, y=y_lines, z=z_lines))
    fig = go.Figure(data = sum([x.data for x in [dot_fig] + line_figs], ()))
    marker_size = 2 if point_labels else 1
    fig.update_traces(marker_size = marker_size)
    
    fig.update_layout(
        width=750,
        height=750,
    )
    fig.show()

In [64]:
# Qingsong's Hamiltonian Path Generator

# get angle of <ABC
def get_angle(p1, base, p2):
    p1, base, p2 = np.array(p1), np.array(base), np.array(p2)
    a = p1 - base
    b = p2 - base
    return np.arccos(np.round(np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)), 5))

def three_points_is_straight(p1, base, p2):
    return np.round(get_angle(p1, base, p2), 5) == np.round(np.pi, 5)

def is_straight(*coords):
    assert len(coords) >= 3
    for i in range(len(coords) - 2):
        if not three_points_is_straight(*coords[i:i + 3]):
            return False
    return True
    

class Graph:
 
    def __init__(self, edges, n): 
        # n is the number of vertices
        # edges is an array of tuples, e.g. [(0, 1), (0, 2)] means there is an edge between 0 and 1 and an edge between 0 and 2

        
        # A 2D array to represent all undirected edges by storing two directed edges for each edge
        self.directed_graph = [[] for _ in range(n)]
        self.n = n
        for (a, b) in edges:
            self.directed_graph[a].append(b) # a connected to b
            self.directed_graph[b].append(a) # b connected to a

def find_all_hamiltonian_paths(graph, start, end, coords, excluded_points = [], max_paths = 5): # undirected_graph
    n = graph.n
        
    # keep track of visted vertices
    visited = [False] * n

    # path starts from v
    path = [start]

    # mark v as visited
    visited[start] = True
    
    paths_found = 0
    
    def hamiltonian(v): # find next vertex to visit, starting from v
        nonlocal paths_found
        n = graph.n
        paths = []
        if paths_found >= max_paths:
            return paths

        # Hamiltonian path exists if all vertices are added to the path
        if len(path) == n - 1 - len(excluded_points):
            if end in graph.directed_graph[v]:# reaches end
                paths.append(path + [end]) # note: + operator creates a copy of path
                show_2d_points(coords, [path + [end]])
                paths_found += 1
            return paths
        
        # else
        for next_v in graph.directed_graph[v]: # check each edge starting from v
            if next_v == end or next_v in excluded_points:
                continue
            
            pre_path_coords = [coords[i] for i in path[-3:]] + [coords[next_v]]

            if len(pre_path_coords) >= 3 and is_straight(*pre_path_coords):
                continue

    
            # if next_v is a new vertex, add next_v to path and run hamiltonian again
            if not visited[next_v]:
                path.append(next_v)
                visited[next_v] = True
                paths += hamiltonian(next_v)
                # If a valid hamiltonian path is found, it will print the path and return
                # If there's no valid solution, it will simply return with no printed message
                # Then we need to explore other possible solutions using backtracking
                # mark next_v as not visited and continue the for loop
                visited[next_v] = False
                path.pop()
        
        return paths

    return hamiltonian(start)

In [65]:
# misc helper functions

def combine_edges(edges):
    merged_this_loop = True
    while merged_this_loop:
        new_edges = []
        merged_this_loop = False
        for edge in edges:
            found_matching = False
            for i, new_edge in enumerate(new_edges):
                if new_edge[0] == new_edge[-1]:
                    continue
                found_matching = True
                if edge[0] == new_edge[-1]:
                    new_edges[i] = new_edges[i] + edge[1:]
                elif edge[-1] == new_edge[0]:
                    new_edges[i] = edge[:-1] + new_edges[i]
                elif edge[0] == new_edge[0]:
                    new_edges[i] = edge[1:][::-1] + new_edges[i]
                elif edge[-1] == new_edge[-1]:
                    new_edges[i] = new_edges[i] + edge[:-1][::-1]
                else:
                    found_matching = False
                    merged_this_loop = True
                if found_matching == True:
                    break
            if not found_matching:
                new_edges.append(edge)
        if len(new_edges) == 1:
            break
        edges = [x.copy() for x in new_edges]
    return new_edges

In [66]:
# Generate Base Triangle
top_left_point = np.array([(3**(1/2))/2, 1/2])
top_right_point = np.array([(-3**(1/2))/2, 1/2])
bottom_point = np.array([0, -1])

base_points = [top_right_point, top_left_point, bottom_point]

show_2d_points(base_points)

In [67]:
# Generate Points for triangle
triangle_coords = []
begin_points = list(zip(
    np.linspace(top_left_point[0], bottom_point[0], TRIANGLE_LENGTH + 1),
    np.linspace(top_left_point[1], bottom_point[1], TRIANGLE_LENGTH + 1))
)
end_points = list(zip(
    np.linspace(top_right_point[0], bottom_point[0], TRIANGLE_LENGTH + 1),
    np.linspace(top_right_point[1], bottom_point[1], TRIANGLE_LENGTH + 1))
)
for layer in range(TRIANGLE_LENGTH + 1):
    begin_point = begin_points[layer]
    end_point = end_points[layer]
    layer_points = list(zip(
        np.linspace(begin_point[0], end_point[0], TRIANGLE_LENGTH + 1 - layer),
        np.linspace(begin_point[1], end_point[1], TRIANGLE_LENGTH + 1 - layer))
    )
    triangle_coords += layer_points[::-1]

triangle_corner_points = [0, TRIANGLE_LENGTH, len(triangle_coords) - 1]

print(triangle_corner_points)

edge_key_1 = (triangle_corner_points[0], triangle_corner_points[1])
edge_key_2 = (triangle_corner_points[2], triangle_corner_points[0])
edge_key_3 = (triangle_corner_points[2], triangle_corner_points[1])

triangle_edge_points = {
    edge_key_1: [1],
    edge_key_2: [len(triangle_coords) - 3],
    edge_key_3: [len(triangle_coords) - 2]
}

for i in range(2, TRIANGLE_LENGTH):
    triangle_edge_points[edge_key_1].append(i)
    triangle_edge_points[edge_key_2].append(triangle_edge_points[edge_key_2][-1] - (i + 1))
    triangle_edge_points[edge_key_3].append(triangle_edge_points[edge_key_3][-1] - i)

print(triangle_edge_points)

show_2d_points(triangle_coords)

[0, 5, 20]
{(0, 5): [1, 2, 3, 4], (20, 0): [18, 15, 11, 6], (20, 5): [19, 17, 14, 10]}


In [68]:
# Generate Edges for triangle

import itertools

# 5 sided triangle
CHOSEN_EDGES = [[11, 15], [15, 18], [16, 18], [13, 16], [12, 13], [1, 7], [7, 12], [1, 2], [2, 8], [8, 9], [9, 10], [10, 14], [14, 17]]

# 9 sided triangle
# CHOSEN_EDGES = [[40, 34], [34, 28], [28, 35], [35, 29], [29, 21], [21, 20], [20, 19], [19, 10], [10, 11], [11, 12], [12, 3], [3, 4], [4, 13], [13, 22], [22, 14], [14, 15], [15, 16], [16, 7], [7, 8], [8, 17], [17, 25], [25, 33], [33, 32], [32, 38], [38, 31], [31, 24], [24, 23], [23, 30], [30, 37], [37, 42], [42, 36], [36, 41], [41, 45], [45, 46], [46, 50], [50, 53], [53, 51], [51, 47], [47, 43], [43, 44], [44, 39]]

def get_neighbors(coords, side_length):
    edges = set()
    def distance(a, b):
        return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5
    neighbor_distance = distance(coords[0], coords[-1])/side_length
    for i, coord_1 in enumerate(coords):
        for j, coord_2 in enumerate(coords):
            if distance(coord_1, coord_2) < neighbor_distance*1.1 and i != j:
                edges.add(frozenset([i, j]))
    return [list(edge) for edge in edges]

def find_gosper_paths(coords, corner_points, edge_points, side_length):

    corner_points = sorted(corner_points)
    neighbors = get_neighbors(coords, side_length)
    graph = Graph(neighbors, len(coords))

    edge_point_count = side_length - 1

    top_edge = edge_points[corner_points[0], corner_points[1]]
    left_edge = edge_points[corner_points[2], corner_points[0]]
    right_edge = edge_points[corner_points[2], corner_points[1]][::-1] # reverse to align w/ left edge
    
    # Create permutations of patterns of edge points to exclude
    exclude_patterns = list(itertools.product([False, True], repeat = edge_point_count//2))

    # Calculate combinations of points that need to be excluded
    exclude_point_groups = []
    for top_exclude_pattern in exclude_patterns:
        for side_exclude_pattern in exclude_patterns:
            exclude_point_group = []
            for i, exclude in enumerate(top_exclude_pattern):
                point_to_exclude = i if exclude else edge_point_count - 1 - i
                exclude_point_group.append(top_edge[point_to_exclude])
            for i, exclude in enumerate(side_exclude_pattern):
                point_to_exclude = i if exclude else edge_point_count - 1 - i
                exclude_point_group.append(left_edge[point_to_exclude])
                exclude_point_group.append(right_edge[point_to_exclude])
            exclude_point_groups.append(exclude_point_group)

    # Calculate edges that triangles will share
    begin_shared_edge = left_edge[edge_point_count//2-1: edge_point_count//2+1]
    end_shared_edge = right_edge[edge_point_count//2-1: edge_point_count//2+1]
    all_paths = []
    max_paths = 10
    for exclude_point_group in exclude_point_groups:
        begin_point = begin_shared_edge[0] if begin_shared_edge[1] in exclude_point_group else begin_shared_edge[1]
        end_point = end_shared_edge[0] if end_shared_edge[1] in exclude_point_group else end_shared_edge[1]
        paths = find_all_hamiltonian_paths(graph, begin_point, end_point, coords, corner_points + exclude_point_group, max_paths)
        full_paths = [[path] + [begin_shared_edge, end_shared_edge] for path in paths]
        all_paths += full_paths
        if len(all_paths) >= max_paths:
            break
    return all_paths

paths = find_gosper_paths(triangle_coords, triangle_corner_points, triangle_edge_points, TRIANGLE_LENGTH)

In [69]:
# Choose edge pattern for icosahedron

show_2d_points(triangle_coords, CHOSEN_EDGES)
CHOSEN_EDGES = paths[6]

In [70]:
# Get Base icosahedron

def icosphere_base():
    # verts for icosahedron
    r = (1.0 + np.sqrt(5.0)) / 2.0;
    verts = np.array([[-1.0, r, 0.0],[ 1.0, r, 0.0],[-1.0, -r, 0.0],
                      [1.0, -r, 0.0],[0.0, -1.0, r],[0.0, 1.0, r],
                      [0.0, -1.0, -r],[0.0, 1.0, -r],[r, 0.0, -1.0],
                      [r, 0.0, 1.0],[ -r, 0.0, -1.0],[-r, 0.0, 1.0]]);
    # rescale the size to radius of 0.5
    verts /= np.linalg.norm(verts[0])

    verts = list(verts)
    
    faces = [[5, 4, 11], [4, 2, 11], [10, 11, 2], [2, 6, 10], 
             [7, 10, 6], [6, 8, 7], [1, 7, 8], [9, 1, 8],
             [3, 9, 8], [8, 6, 3], [6, 2, 3], [2, 4, 3], 
             [9, 3, 4], [4, 5, 9], [1, 9, 5], [5, 0, 1], 
             [7, 1, 0], [10, 7, 0], [11, 10, 0], [0, 5, 11]]
                
    return np.array(verts), faces

In [71]:
# Find Transformation Matrices

# use a scalar to scale 
# http://nghiaho.com/?page_id=671
def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = centroid_B

    return R, t

triangle_mat = np.array(base_points)
triangle_mat = np.hstack([triangle_mat, np.zeros((3, 1))])

icosphere_coords, icosphere_faces = icosphere_base()

transformation_mats_list = []

for face in icosphere_faces:
    icosphere_mat = np.array([icosphere_coords[i] for i in face])
    rotation_mat, translation_mat = rigid_transform_3D(triangle_mat.T, icosphere_mat.T)
    scale_factor = np.linalg.norm(icosphere_mat[0] - np.mean(icosphere_mat, axis=0))
    transformation_mats_list.append({
        'rotation_mat': rotation_mat,
        'translation_mat': translation_mat,
        'scale_factor': scale_factor
    })

def apply_transform(mat, tf_mats):
    rotated = tf_mats['rotation_mat'] @ np.array(mat).T
    scaled = rotated * tf_mats['scale_factor']
    translated = scaled + np.tile(tf_mats['translation_mat'], (1, mat.shape[0])) 
    return translated.T

det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...
det(R) < R, reflection detected!, correcting for it ...


In [72]:
# Map patterns to icosahedron

icosphere_coords, icosphere_faces = icosphere_base()

# to make it a list instead of an np array
icosphere_coords = [np.array(x) for x in icosphere_coords]

triangle_coords_3d = np.pad(triangle_coords, ((0, 0), (0, 1)))

edge_points = {}

icosphere_edges = []

for i, face in enumerate(icosphere_faces):
    transformed_coords = apply_transform(triangle_coords_3d, transformation_mats_list[i])
    base_to_new = [None] * len(triangle_coords)

    # Match corner points
    for j in range(3):
        base_to_new[triangle_corner_points[j]] = face[j]
    
    # Match edges or add new ones
    for edge, points in triangle_edge_points.items():
        new_edge = (base_to_new[edge[0]], base_to_new[edge[1]])
        if new_edge in edge_points:
            new_points = edge_points[new_edge]
            for j in range(len(edge_points[new_edge])):
                base_to_new[points[j]] = new_points[j]
        elif new_edge[::-1] in edge_points:
            new_points = edge_points[new_edge[::-1]][::-1]
            for j in range(len(edge_points[new_edge[::-1]])):
                base_to_new[points[j]] = new_points[j]
        else:
            edge_points[new_edge] = []
            for point in points:
                icosphere_coords.append(transformed_coords[point])
                base_to_new[point] = len(icosphere_coords) - 1
                edge_points[new_edge].append(base_to_new[point])

    # Match rest of points
    for j in range(len(triangle_coords)):
        if base_to_new[j] == None:
            icosphere_coords.append(transformed_coords[j])
            base_to_new[j] = len(icosphere_coords) - 1

    # Create edges
    for edge_list in CHOSEN_EDGES:
        new_edges = [base_to_new[point] for point in edge_list]
        if new_edges not in icosphere_edges and new_edges[::-1] not in icosphere_edges:
            icosphere_edges.append(new_edges)

icosphere_edges = combine_edges(icosphere_edges)

angle = -30 * np.pi/180
rotation_mat = np.array([
    [1, 0, 0],
    [0, np.cos(angle), -np.sin(angle)], 
    [0, np.sin(angle), np.cos(angle)]
])
 
icosphere_coords = (rotation_mat @ np.array(icosphere_coords).T).T

show_3d_points(icosphere_coords, icosphere_edges, False)

In [73]:
# Make jipcad code

with open(f'nom/icosahedron_{TRIANGLE_LENGTH}.nom', 'w') as file:

    file.write('''
bank b
    set sphere_radius 1 0.1 3 0.1
    set sweep_radius 0.05 0.01 0.5 0.01
    set sweep_twist 0 0 360 1
    set sweep_azimuth 0 0 360 1
    set cross_section_sides 10 5 20 1
    list (sphere_radius sweep_radius sweep_twist sweep_azimuth cross_section_sides)
endbank
''')

    for i, point in enumerate(icosphere_coords):
        file.write('point pt_{} ({{expr {}*$b.sphere_radius}} {{expr {}*$b.sphere_radius}} {{expr {}*$b.sphere_radius}}) endpoint\n'.format(i, point[0], point[1], point[2]))
    file.write('\n')

    for i, edge in enumerate(icosphere_edges):
        file.write('polyline line_{} ('.format(i))
        file.write(' '.join(['pt_{}'.format(pt) for pt in edge[:-1]]))
        if edge[-1] == edge[0]:
            file.write(') closed')
        else:
            file.write(' pt_{})'.format(edge[-1]))
        file.write(' endpolyline\n')
    file.write('\n')

    # CROSS SECTION
    file.write('circle ngon ( $b.sweep_radius $b.cross_section_sides ) endcircle\n')

    #instance ngon_insance ngon endinstance # debug

    # SWEEP
    for i, edge in enumerate(icosphere_edges):
        file.write('''sweep sweep_{}
        crosssection ngon begincap endcap endcrosssection
        path line_{} twist $b.sweep_twist azimuth $b.sweep_azimuth mintorsion endpath
    endsweep\n'''.format(i, i))
        file.write('instance inst_{} sweep_{} endinstance\n'.format(i, i))
    file.write('\n')
